In [1]:
import h3
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
# read geojson file
f = open('../gangnamGu.geojson')
data = json.load(f)
f.close()

In [3]:
# because h3 get coordinates as reversed order of geojson file. I have to reverse the order of coordinates prior to get hexagons
new_coords = [[coord[1],coord[0]] for coord in data['features'][0]['geometry']['coordinates'][0]]

In [4]:
# generate a new geojson file with the reversed coordinates
new_geojson = {'type':'Polygon', 'coordinates':[new_coords]}

In [6]:
# get indices of h3 hexagons from a new geojson 
hexagons = list(h3.polyfill(new_geojson, res=8))
print(hexagons)

['8830e1ca31fffff', '8830e1cae9fffff', '8830e1ca23fffff', '8830e1c853fffff', '8830e1c81bfffff', '8830e1ca51fffff', '8830e1c857fffff', '8830e1c8e3fffff', '8830e1ca2bfffff', '8830e1ca37fffff', '8830e1ca17fffff', '8830e1c843fffff', '8830e1c85dfffff', '8830e1c80bfffff', '8830e1c887fffff', '8830e1c8c1fffff', '8830e1ca3dfffff', '8830e1c8c7fffff', '8830e1ca41fffff', '8830e1c125fffff', '8830e1c8ebfffff', '8830e1c855fffff', '8830e1ca35fffff', '8830e1c813fffff', '8830e1cae1fffff', '8830e1c885fffff', '8830e1c8cdfffff', '8830e1c8c9fffff', '8830e1caa3fffff', '8830e1c859fffff', '8830e1ca4bfffff', '8830e1c8e9fffff', '8830e1ca55fffff', '8830e1c8cbfffff', '8830e1caa1fffff', '8830e1cae5fffff', '8830e1ca15fffff', '8830e1ca33fffff', '8830e1cac5fffff', '8830e1c851fffff', '8830e1c88dfffff', '8830e1c8e1fffff', '8830e1ca01fffff', '8830e1c84bfffff', '8830e1ca09fffff', '8830e1ca53fffff', '8830e1caa5fffff', '8830e1c85bfffff', '8830e1c81dfffff', '8830e1ca11fffff', '8830e1cae7fffff', '8830e1c811fffff', '8830e1c889

In [7]:
# get centerpoints from the indices
hexagons_centerpoints = [[index, h3.h3_to_geo(index)[1],h3.h3_to_geo(index)[0]] for index in hexagons]
print(hexagons_centerpoints)

[['8830e1ca31fffff', 127.04688904729679, 37.50081662332127], ['8830e1cae9fffff', 127.06115724384675, 37.51472424327642], ['8830e1ca23fffff', 127.03581011702948, 37.493646045460196], ['8830e1c853fffff', 127.07212309912045, 37.48908232833551], ['8830e1c81bfffff', 127.08474117007685, 37.48321129859285], ['8830e1ca51fffff', 127.03119728694772, 37.53274814680687], ['8830e1c857fffff', 127.06893796974761, 37.482346846029735], ['8830e1c8e3fffff', 127.10678904551656, 37.46472558325376], ['8830e1ca2bfffff', 127.03109316184022, 37.499944819235886], ['8830e1ca37fffff', 127.05160593525254, 37.494516323639964], ['8830e1ca17fffff', 127.05325484478125, 37.514288432163546], ['8830e1c843fffff', 127.05313978103156, 37.4814805437727], ['8830e1c85dfffff', 127.05632233579533, 37.488215424462304], ['8830e1c80bfffff', 127.06575367468858, 37.47561182843134], ['8830e1c887fffff', 127.12259954912638, 37.46558267707841], ['8830e1c8c1fffff', 127.10054938199883, 37.48407389952007], ['8830e1ca3dfffff', 127.0389904777

In [8]:
hexagons_polygons =[Polygon([(coords[1], coords[0]) for coords in h3.h3_to_geo_boundary(index)]) for index in hexagons]
print(hexagons_polygons[:5])

[<shapely.geometry.polygon.Polygon object at 0x000002782B613CA0>, <shapely.geometry.polygon.Polygon object at 0x000002782B613E20>, <shapely.geometry.polygon.Polygon object at 0x0000027828C768B0>, <shapely.geometry.polygon.Polygon object at 0x0000027828C76CA0>, <shapely.geometry.polygon.Polygon object at 0x0000027828C76610>]


In [9]:
# create a dataframe with the indices and centerpoints
df_hexagons = pd.DataFrame(hexagons_centerpoints, columns =['index', 'x','y'])

In [10]:
df_hexagons.head()

,index,x,y
0,8830e1ca31fffff,127.046889,37.500817
1,8830e1cae9fffff,127.061157,37.514724
2,8830e1ca23fffff,127.035810,37.493646
3,8830e1c853fffff,127.072123,37.489082
4,8830e1c81bfffff,127.084741,37.483211


In [11]:
# convert the dataframe into a geodataframe with geometries
gdf_hexagons = gpd.GeoDataFrame(df_hexagons, geometry = hexagons_polygons)

In [12]:
gdf_hexagons.head()

,index,x,y,geometry
0,8830e1ca31fffff,127.046889,37.500817,"POLYGON ((127.05109 37.49886, 127.05058 37.503..."
1,8830e1cae9fffff,127.061157,37.514724,"POLYGON ((127.06536 37.51277, 127.06485 37.517..."
2,8830e1ca23fffff,127.035810,37.493646,"POLYGON ((127.04001 37.49169, 127.03950 37.496..."
3,8830e1c853fffff,127.072123,37.489082,"POLYGON ((127.07633 37.48713, 127.07582 37.491..."
4,8830e1c81bfffff,127.084741,37.483211,"POLYGON ((127.08895 37.48125, 127.08844 37.485..."


In [13]:
# export the geodataframe into a geojson file 
gdf_hexagons.to_file('../hexagons.geojson', encoding='utf8', driver='GeoJSON')